# AgentTesla
> Some dot net exploration featuring agenttesla

- toc: true 
- badges: true
- categories: [dotnet,python,agenttesla,research]


## Overview
Our goal is to write a static strings table decryptor for this AgentTesla variant. The strings are stored in a giant INT array so our goals are twofold. 
- statically identify .NET code sections using opcodes and extract some values from the code
- statically identify the location of large int arrays in .NET binaries

The obfuscator used is called **obfuscar** and it looks to be responsible for the strings encryption [ref](https://github.com/obfuscar/obfuscar/blob/d11f455af81a20209a0d746077447f50f7074f92/Obfuscar/Obfuscator.cs#L1370).

### Samples (unpacked)
- `20f4ec03549be469302c0fcb5f55307680fd189aa733f90eb59cb2fbc34317cc` [malshare](https://malshare.com/sample.php?action=detail&hash=20f4ec03549be469302c0fcb5f55307680fd189aa733f90eb59cb2fbc34317cc)
- `cb3afdb1e17d5bdaf641e633434ac71855e5dcfdd21d66a565f0dc9844d30030` [malshare](https://malshare.com/sample.php?action=detail&hash=cb3afdb1e17d5bdaf641e633434ac71855e5dcfdd21d66a565f0dc9844d30030)

### References
- [dotnetfile](https://github.com/pan-unit42/dotnetfile)
- [dnfile](https://github.com/malwarefrank/dnfile)
- [dncil](https://github.com/mandiant/dncil)

## Analysis

The string location function.
![](https://i.imgur.com/f7b9k0o.png)

The string decryption routine.

In [3]:
data=[152,155,209,208,215,214,129,224,239,142,196,197,134,239,236,159,215,214,130,202,205,198,197,196,203,236,253,252,233,211,208,234,194,195,215,228,227,208,0xff,254,190]
out = ''
for i in range(len(data)):
    out += chr(data[i] ^ (i & 0xff) ^ 170)

out



'20yyyy-MM-dd HH:mm:ssyyyy_MM_dd_HH_mm_ss<'

### Locate The Array

#### Byte Matching Regex To Locate The Array
We could possibly locate the array based on the fact that it is huge! Not many giant arrays in this malware.
```
04 20 B1 2E 00 00 //ldc.i4    11953
8D 2B 00 00 01   // newarr    [mscorlib]System.Byte
```
The drawbacks are that we will still need to parse the .NET to find the token with the data offset to the actual array being passed to `RuntimeHelpers::InitializeArray`.

#### .NET Parsing To Locate The Array


In [12]:
TARGET_PATH = '/tmp/at.bin'

file_data = open(TARGET_PATH,'rb').read()


In [38]:
import pefile
import sys, struct, clr
clr.AddReference("System.Memory")
from System.Reflection import Assembly, MethodInfo, BindingFlags
from System import Type

DNLIB_PATH = '/tmp/dnlib.dll'
clr.AddReference(DNLIB_PATH)

import dnlib
from dnlib.DotNet import *
from dnlib.DotNet.Emit import OpCodes

module = dnlib.DotNet.ModuleDefMD.Load(TARGET_PATH)

In [42]:




for mtype in module.GetTypes():
    if not mtype.HasMethods:
        continue
    for method in mtype.Methods:
        if not method.HasBody: 
            continue
        if not method.Body.HasInstructions: 
            continue
        if len(method.Body.Instructions) < 20:
            continue
        key_set = False 
        block_set = False
        for ptr in range(20):
            if "RuntimeHelpers::InitializeArray" in method.Body.Instructions[ptr].ToString():
                arr_inst = method.Body.Instructions[ptr-1]
                break
                
                
dir(arr_inst)

['CalculateStackUsage',
 'Clone',
 'Create',
 'CreateLdcI4',
 'Equals',
 'Finalize',
 'GetArgumentType',
 'GetHashCode',
 'GetLdcI4Value',
 'GetLocal',
 'GetParameter',
 'GetParameterIndex',
 'GetSize',
 'GetType',
 'IsBr',
 'IsBrfalse',
 'IsBrtrue',
 'IsConditionalBranch',
 'IsLdarg',
 'IsLdcI4',
 'IsLdloc',
 'IsLeave',
 'IsStarg',
 'IsStloc',
 'MemberwiseClone',
 'Offset',
 'OpCode',
 'Operand',
 'Overloads',
 'ReferenceEquals',
 'SequencePoint',
 'ToString',
 'UpdateStack',
 '__call__',
 '__class__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__overloads__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [29]:
dir(arr_inst.Operand)

['Access',
 'Attributes',
 'Constant',
 'CustomAttributes',
 'CustomDebugInfos',
 'DeclaringType',
 'DeclaringType2',
 'ElementType',
 'Equals',
 'FieldOffset',
 'FieldSig',
 'FieldType',
 'Finalize',
 'FullName',
 'GetConstant_NoLock',
 'GetFieldOffset_NoLock',
 'GetFieldSize',
 'GetHashCode',
 'GetImplMap_NoLock',
 'GetInitialValue_NoLock',
 'GetMarshalType_NoLock',
 'GetRVA_NoLock',
 'GetType',
 'HasConstant',
 'HasConstantTag',
 'HasCustomAttributeTag',
 'HasCustomAttributes',
 'HasCustomDebugInformationTag',
 'HasCustomDebugInfos',
 'HasDefault',
 'HasFieldMarshal',
 'HasFieldMarshalTag',
 'HasFieldRVA',
 'HasImplMap',
 'HasLayoutInfo',
 'HasMarshalType',
 'ImplMap',
 'InitialValue',
 'InitializeCustomAttributes',
 'InitializeCustomDebugInfos',
 'IsAssembly',
 'IsCompilerControlled',
 'IsFamily',
 'IsFamilyAndAssembly',
 'IsFamilyOrAssembly',
 'IsInitOnly',
 'IsLiteral',
 'IsNotSerialized',
 'IsPinvokeImpl',
 'IsPrivate',
 'IsPrivateScope',
 'IsPublic',
 'IsRuntimeSpecialName',
 '

In [36]:
print(hex(arr_inst.Operand.RVA))
print(hex(arr_inst.Operand.GetFieldSize()))

0x260b8
0x2eb1


TypeError: 'NoneType' object is not callable

In [45]:
pe = pefile.PE(data=file_data, fast_load=True)

hex(pe.get_offset_from_rva(0x260b8))


'0x242b8'

In [77]:
target_module = dnlib.DotNet.ModuleDefMD.Load(TARGET_PATH)


def pct_ascii(s):
    return len([c for c in s if c < 128 or c == 0]) / len(s)


def decrypt(data, key):
    out = []
    for i in range(len(data)):
        out.append((data[i] ^ i ^ key) & 0xff)
    return bytes(out)


def get_strings_table(target_module, pe):
    out = []
    keys = []
    for mtype in target_module.GetTypes():
        if not mtype.HasMethods:
            continue
        for method in mtype.Methods:
            # The string decryption happens in a constructor
            if not method.IsConstructor:
                continue
            if not method.HasBody: 
                continue
            if not method.Body.HasInstructions: 
                continue
            if len(method.Body.Instructions) < 30:
                continue
            key_set = False 
            block_set = False
            key_flag = False
            for ptr in range(30):
                if "RuntimeHelpers::InitializeArray" in method.Body.Instructions[ptr].ToString():
                    arr_inst = method.Body.Instructions[ptr-1]
                    arr_rva = arr_inst.Operand.RVA
                    arr_size = arr_inst.Operand.GetFieldSize()
                    out.append(pe.get_data(arr_rva, arr_size))
                    key_flag = True
                if key_flag:
                    if "xor" in method.Body.Instructions[ptr].ToString() and "ldc.i4" in method.Body.Instructions[ptr - 1].ToString():
                        keys.append(method.Body.Instructions[ptr - 1].Operand)
    if len(out) == 0:
        return None
    arr_data = max(out, key=len)
    # For each key try to decrypt and save the one that
    # decrypt to valid ascii
    ptxt_data = None
    for key in keys:
        tmp_out = decrypt(arr_data, key)
        if pct_ascii(tmp_out) > 0.8:
            ptxt_data = tmp_out
            break
    return ptxt_data


strings_table = get_strings_table(target_module, pe)


strings_table



b'20yyyy-MM-dd HH:mm:ssyyyy_MM_dd_HH_mm_ss<br><hr>ObjectLengthChainingModeGCMAuthTagLengthChainingModeKeyDataBlobAESMicrosoft Primitive ProviderCONNECTIONKEEP-ALIVEPROXY-AUTHENTICATEPROXY-AUTHORIZATIONTETRAILERTRANSFER-ENCODINGUPGRADE%startupfolder%\\%insfolder%\\%insname%/\\%insfolder%\\Software\\Microsoft\\Windows\\CurrentVersion\\Run%insregname%SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Explorer\\StartupApproved\\RunTrue%GETMozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0OKhttp://pjQxEo.com\\XPXSELECT * FROM Win32_ProcessorName MBUnknownCOCO_-_.zip yyyy-MM-dd hh-mm-ssCookieapplication/zipSCSC_.jpegScreenshotimage/jpeg/log.tmpKLKL_.html<html></html>Logtext/html[]Time: MM/dd/yyyy HH:mm:ssUser Name: Computer Name: OSFullName: CPU: RAM: IP Address: New  Recovered!User Name: OSFullNameuninstallSoftware\\Microsoft\\Windows NT\\CurrentVersion\\WindowsLoad%ftphost%/%ftpuser%%ftppassword%STORLengthWriteCloseGetBytesOpera BrowserOpera Software\\Opera StableYan

In [ ]:
# 0x0001F0C6 20A7000000   */ IL_000F: ldc.i4    167   // index
# 0x0001F0CB 202D080000   */ IL_0014: ldc.i4    2093 // offset
# 0x0001F0D0 1F09         */ IL_0019: ldc.i4.s  9    // size
# 0x0001F0D2 282D020006   */ IL_001B: call      string


In [94]:

# offset,size
str_offsets = []
for mtype in target_module.GetTypes():
    if not mtype.HasMethods:
        continue
    for method in mtype.Methods:
        # The get string functions are public and return a string
        if not method.IsPublic:
            continue
        if method.ReturnType.ToString() != "System.String":
            continue
        if not method.HasBody: 
            continue
        if not method.Body.HasInstructions: 
            continue
        if len(method.Body.Instructions) < 10:
            continue
        key_set = False 
        block_set = False
        key_flag = False
        for ptr in range(10):
            if "call System.String" in method.Body.Instructions[ptr].ToString():
                if "ldc" in method.Body.Instructions[ptr-1].ToString() and \
                "ldc" in method.Body.Instructions[ptr-2].ToString() and \
                "ldc" in method.Body.Instructions[ptr-3].ToString():
                    if method.Body.Instructions[ptr-1].Operand is None:
                        str_size = int(method.Body.Instructions[ptr-1].ToString().split('.')[-1])
                    else:
                        str_size = method.Body.Instructions[ptr-1].Operand
                    if method.Body.Instructions[ptr-2].Operand is None:
                        str_offset = int(method.Body.Instructions[ptr-2].ToString().split('.')[-1])
                    else:
                        str_offset = method.Body.Instructions[ptr-2].Operand
                    str_offsets.append((str_offset, str_size))
str_offsets

[(0, 0),
 (0, 2),
 (2, 19),
 (21, 19),
 (40, 4),
 (44, 4),
 (48, 12),
 (60, 15),
 (75, 13),
 (88, 12),
 (100, 11),
 (111, 3),
 (114, 28),
 (142, 10),
 (152, 10),
 (162, 18),
 (180, 19),
 (199, 2),
 (201, 7),
 (208, 17),
 (225, 7),
 (232, 15),
 (247, 22),
 (269, 1),
 (270, 13),
 (283, 45),
 (328, 12),
 (340, 70),
 (410, 4),
 (414, 1),
 (415, 3),
 (418, 78),
 (496, 2),
 (498, 17),
 (515, 4),
 (519, 29),
 (548, 4),
 (552, 3),
 (555, 7),
 (562, 2),
 (564, 3),
 (567, 1),
 (568, 1),
 (569, 4),
 (573, 1),
 (574, 19),
 (593, 6),
 (599, 15),
 (614, 2),
 (616, 3),
 (619, 5),
 (624, 10),
 (634, 10),
 (644, 8),
 (652, 2),
 (654, 3),
 (657, 5),
 (662, 6),
 (668, 7),
 (675, 3),
 (678, 9),
 (687, 1),
 (688, 1),
 (689, 6),
 (695, 19),
 (714, 11),
 (725, 15),
 (740, 12),
 (752, 5),
 (757, 5),
 (762, 12),
 (774, 4),
 (778, 11),
 (789, 9),
 (798, 2),
 (800, 10),
 (810, 9),
 (819, 52),
 (871, 4),
 (875, 10),
 (885, 9),
 (894, 13),
 (907, 4),
 (911, 6),
 (917, 5),
 (922, 5),
 (927, 8),
 (935, 13),
 (948, 2

In [95]:
strings = []
for offset_info in str_offsets:
    strings.append(strings_table[offset_info[0]:offset_info[0]+offset_info[1]])

strings

[b'',
 b'20',
 b'yyyy-MM-dd HH:mm:ss',
 b'yyyy_MM_dd_HH_mm_ss',
 b'<br>',
 b'<hr>',
 b'ObjectLength',
 b'ChainingModeGCM',
 b'AuthTagLength',
 b'ChainingMode',
 b'KeyDataBlob',
 b'AES',
 b'Microsoft Primitive Provider',
 b'CONNECTION',
 b'KEEP-ALIVE',
 b'PROXY-AUTHENTICATE',
 b'PROXY-AUTHORIZATION',
 b'TE',
 b'TRAILER',
 b'TRANSFER-ENCODING',
 b'UPGRADE',
 b'%startupfolder%',
 b'\\%insfolder%\\%insname%',
 b'/',
 b'\\%insfolder%\\',
 b'Software\\Microsoft\\Windows\\CurrentVersion\\Run',
 b'%insregname%',
 b'SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Explorer\\StartupApproved\\Run',
 b'True',
 b'%',
 b'GET',
 b'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0',
 b'OK',
 b'http://pjQxEo.com',
 b'\\XPX',
 b'SELECT * FROM Win32_Processor',
 b'Name',
 b' MB',
 b'Unknown',
 b'CO',
 b'CO_',
 b'-',
 b'_',
 b'.zip',
 b' ',
 b'yyyy-MM-dd hh-mm-ss',
 b'Cookie',
 b'application/zip',
 b'SC',
 b'SC_',
 b'.jpeg',
 b'Screenshot',
 b'image/jpeg',
 b'/log.tmp',
 b'KL',
 